In [2]:
import sys
sys.path.insert(0, '../')

# sys.path.remove("/home/users/yixiuz/.local/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/bin")

In [6]:
from importlib import reload

In [57]:
reload(dataset_utils)
reload(datasets)
reload(countdown)
reload(losses_utils)
reload(losses)
reload(model_utils)
reload(models)

<module 'lib.models.models' from '../lib/models/models.py'>

In [58]:
import torch
import torch.nn as nn
import ml_collections
import yaml
import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import torch.utils.tensorboard as tensorboard
from tqdm import tqdm
import sys
import signal
import argparse


import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.datasets as datasets
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.loggers.loggers as loggers
import lib.loggers.logger_utils as logger_utils

import config.train.countdown as countdown
cfg = countdown.get_config()

custom_name = None

print("Training with config", cfg.experiment_name)

preempted_path = Path("null")
if cfg.saving.enable_preemption_recovery:

    preempted_path = bookkeeping.check_for_preempted_run(cfg.save_location,
        cfg.saving.preemption_start_day_YYYYhyphenMMhyphenDD,
        cfg,
        cfg.saving.prepare_to_resume_after_timeout
    )

if preempted_path.as_posix() == "null":
    save_dir, checkpoint_dir, config_dir = \
        bookkeeping.create_experiment_folder(
            cfg.save_location,
            cfg.experiment_name if custom_name is None else custom_name,
            custom_name is None
    )
    bookkeeping.save_config_as_yaml(cfg, config_dir)

    # bookkeeping.save_git_hash(save_dir)

else:
    print("Resuming from preempted run: ", preempted_path)
    save_dir = preempted_path
    checkpoint_dir, config_dir = bookkeeping.create_inner_experiment_folders(save_dir)

writer = bookkeeping.setup_tensorboard(save_dir, 0)

device = torch.device(cfg.device)

model = model_utils.create_model(cfg, device)
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

dataset = dataset_utils.get_dataset(cfg, device)
dataloader = torch.utils.data.DataLoader(dataset,
    batch_size=cfg.data.batch_size,
    shuffle=cfg.data.shuffle)

loss = losses_utils.get_loss(cfg)

training_step = training_utils.get_train_step(cfg)

optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)

state = {
    'model': model,
    'optimizer': optimizer,
    'n_iter': 0
}

bookkeeping.setup_preemption(save_dir, checkpoint_dir, state,
    cfg.saving.num_checkpoints_to_keep,
    cfg.saving.prepare_to_resume_after_timeout)


if not preempted_path.as_posix() == 'null':
    state = bookkeeping.resume_training(preempted_path, state, cfg.device)

low_freq_loggers = []
for logger in cfg.saving.low_freq_loggers:
    low_freq_loggers.append(logger_utils.get_logger(logger))

exit_flag = False


while True:
    for minibatch in tqdm(dataloader):

        training_step.step(state, minibatch, loss, writer)

        if state['n_iter'] % cfg.saving.checkpoint_freq == 0 or state['n_iter'] == cfg.training.n_iters-1:
            bookkeeping.save_checkpoint(checkpoint_dir, state,
                cfg.saving.num_checkpoints_to_keep)

        if state['n_iter'] % cfg.saving.log_low_freq == 0 or state['n_iter'] == cfg.training.n_iters-1:
            for logger in low_freq_loggers:
                logger(state=state, cfg=cfg, writer=writer,
                       minibatch=minibatch, dataset=dataset)

        state['n_iter'] += 1
        if state['n_iter'] > cfg.training.n_iters - 1:
            exit_flag = True
            break

    if exit_flag:
        break

writer.close()

return save_dir

Training with config countdown_test
number of parameters:  5318433


  0%|          | 0/100 [00:00<?, ?it/s]


AssertionError: 

In [53]:
def check(x):
    return x.isinf().any() or x.isnan().any()

In [ ]:
%debug

> /home/groups/swl1/yixiuz/torch_fid/tauLDR/lib/losses/losses.py(231)calc_loss()
    229 
    230         assert(not torch.isnan(outer_sum_sig).any())
--> 231         assert(not torch.isnan(sig_norm).any())
    232 
    233         sig_mean = torch.mean(- outer_sum_sig/sig_norm)

ipdb> sig_norm.shape
torch.Size([64])
ipdb> sig_norm
tensor([0.8919, 1.0954, 0.9283, 0.8640, 0.8165, 0.7595, 1.2708, 1.5399, 0.9327,
        1.1924, 0.7996, 1.1691, 0.9891, 1.1607, 0.9468, 1.0328, 0.9492, 0.8983,
        0.8684, 1.1195, 0.9773, 1.1841, 1.4582, 1.0226, 1.4081, 1.6932, 1.0802,
        0.9911, 1.2367, 0.6933, 1.2016, 0.8028, 1.3088, 0.9708, 1.0937, 0.7423,
        0.8969, 1.2378, 0.9822, 1.0552, 1.0677, 1.1103, 1.0952, 1.1112, 1.3776,
        1.0496, 1.0570, 1.2912, 1.0259, 1.0351, 1.0888, 1.0440, 1.0256, 0.8169,
        0.9337, 0.8716, 1.0981, 1.1817, 0.7441, 0.9565, 0.9031,    nan, 0.7473,
           nan], device='cuda:0')
ipdb> rate_sig_norm
tensor([[[  4.6765,   4.6765,   4.6765,  ...,   4.67

In [49]:
model(minibatch, torch.ones((cfg.data.batch_size,)).to(device) * 0.1).isinf().any()

tensor(False, device='cuda:0')